# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [110]:
# skopiuj pierścień ilorazowy wielomianów z pierwszych zajęć
import numpy as np
from copy import deepcopy



def create_poly(table):
    if isinstance(table,Poly):
        return table
    if isinstance(table, list):
        return Poly(table)
    if isinstance(table, (float, int)):
        return Poly([table])
    if isinstance(table, ZnPoly):
        print("test")
    raise ValueError(f"unknown table type: {table}")

class Poly:
    def __init__(self, coef, variable = None):
        self.coef = coef
        self.variable = variable
        if self.variable is None : self.variable = 'x'
        if len(self.variable) > 1 : self.variable = f"({self.variable})"

    def normalize(self) :
        while self.coef[-1] == 0 and len(self.coef) > 1 :
            self.coef.pop()

    def __iter__(self):
        self.current = 0
        return self

    def __next__(self):
        if self.current < len(self.coef):
            item = self.coef[self.current]
            self.current += 1
            return item
        else:
            raise StopIteration


    def __getitem__(self, index):
        if isinstance(index, slice) :
            return Poly(self.coef[index], self.variable)
        return self.coef[index]

    def __setitem__(self, index, value):
        if isinstance(index, slice) :
            if isinstance(value, Poly):
                self.coef[index] = value.coef
            else:
                self.coef[index] = value
        else:
            self.coef[index] = value

    def __len__(self):
        return len(self.coef)
    
    def __add__(self, poly2) :
        return Poly(np.polyadd(self.coef, poly2.coef))
    
    def __mul__(poly1, poly2) :   
        return Poly(np.polymul(poly1.coef, poly2.coef))
    
    def __rmul__(self, other):
        return self * other
    
    def __radd__(self, other):
        return self + other
    
    def __rsub__(self, other) :
        return self - other
    
    def __sub__(poly1, poly2) :
        return poly1 + (poly2 * -1)
    
    def __truediv__(poly1, poly2) :
        poly1 = create_poly(poly1)
        poly2 = create_poly(poly2)
        
        return np.polydiv(poly1,poly2)
            
    def __floordiv__(poly1, poly2) :
        return (poly1.__truediv__(poly2))[0]
    
    def __mod__(poly1, poly2) :
        return (poly1.__truediv__(poly2))[1]
    
    def __eq__(poly1, poly2) :
        if not (isinstance(poly1, Poly) and isinstance(poly2, Poly)) : return False
        check1 = deepcopy(poly1)
        check2 = deepcopy(poly2)
        check1.normalize()
        check2.normalize()
        
        if len(check1) != len(check2) : return False

        for i in range(len(check1)) :
            if check1[i] != check2[i] : return False

        return True  

    def get_coef(self, k):
        n = len(self.coef)
        if k < n:
            return [0] * (k - n + 1) + self.coef
        else:
            return [0] * (k - n + 1) + self.coef    

            
    def convert_to_nice(self,val,i):
        if val == 0:
            return ""
        if self.first_print:
            symbol = "-" if val < 0 else ""
            self.first_print = False
        else:
            symbol = " - " if val < 0 else " + "

        
        if len(self.coef) - i - 1 == 0:
            return symbol + str(abs(val))
        
        result = f"x^{len(self.coef) - i - 1}"


        if abs(val) == 1:
            return symbol + result
        else:
            return symbol + str(val) + result

    def __str__(self):
        self.first_print = True
        return f' {"".join(self.convert_to_nice(val,i) for i, val in enumerate(self.coef))}'
    def __repr__(self):
        self.first_print = True
        return f'{"".join(self.convert_to_nice(val,i) for i, val in enumerate(self.coef))}'


class ZnWRing:
    def __init__(self, N, W):
        self.N = N
        self.W = self.moderate(W)

    def fix_float(self, number) :
        return float(number) if not float(number).is_integer() else int(number)
    
    def moderate(self, poly) :
        return Poly([self.fix_float(x % self.N) for x in poly])
    
    def repair(self, poly) :
        
        if isinstance(poly, int) or isinstance(poly, float) : poly = poly % self.N
        else : poly = self.moderate((poly % self.W))

        poly2 = []
        for number in poly:
            poly2.append(self.fix_float(number))
        return Poly(poly2)
        
    def add(self, poly1, poly2) :
        return self.repair(poly1 + poly2)
    
    def subtract(self, poly1, poly2) :
        return self.repair(poly1 - poly2)
    
    def mul(self, poly1, poly2) :
        return self.repair(poly1 * poly2)

    def div(self, poly1, poly2) :
        return self.repair((poly1 / poly2)[1])
    
    
    def __str__(self) :
        return repr(self)
    
    def __repr__(self) :
        return f"N = {self.N}\nW = " + str(self.W)
    
    def __eq__(self, other):
        if not isinstance(other, ZnWRing):
            return ValueError("Must be a ZnWRing!")
        return self.N == other.N and self.W == other.W


class ZnPoly:
    def __init__(self, poly:Poly, N = 17, W = [1,0,0,0,1]):
        self.ZnWRing = ZnWRing(N,W)
        self.poly = self.ZnWRing.repair(create_poly(poly))
    
    def __mul__(self,other):
        if not isinstance(other, ZnPoly):
            return ZnPoly(self.ZnWRing.mul(self.poly,create_poly(other)))
        return ZnPoly(self.ZnWRing.mul(self.poly,other.poly))

    def __rmul__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return self.__mul__(other)
    
    def __add__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly(self.ZnWRing.add(self.poly, other.poly))
    
    def __radd__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return self.__add__(other)
    
    def __sub__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly(self.ZnWRing.add(self.poly, other.poly))
    
    def __rsub__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly(self.ZnWRing.add(other.poly, self.poly))

    def __len__(self):
        return len(self.poly)

    def __eq__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly.__eq__(self.poly, other.poly) and self.ZnWRing == other.ZnWRing
    
    def __repr__(self):
        return str(self.poly)
    
    def __str__(self):
        return self.__repr__()

#======================== TESTS ======================================

w1 = [7,0,0,14,0,0,0]
w2 = [0,12,10,6]
w3 = [1,1,11,7]

w1 = ZnPoly(w1)
w2 = ZnPoly(w2)
w3 = ZnPoly(w3)

print(f"w1 = {w1}")
print(f"w2 = {w2}")
print(f"w3 = {w3}")

print(f"w1 + w2 = {w1 + w2}")
print(f"w1 * w2 = {w1 * w2}")

w1 =  14x^3 + 10x^2
w2 =  12x^2 + 10x^1 + 6
w3 =  x^3 + x^2 + 11x^1 + 7
w1 + w2 =  14x^3 + 5x^2 + 10x^1 + 6
w1 * w2 =  14x^3 + 9x^2 + 2x^1 + 12


## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

In [115]:
K = 2
q = 17

#pierścień Z17[X]/X^4 + 1

prop = 0.05

n = 4

### Generowanie klucza

jak ja się ciesze że jest napisane która rzecz jest wymiaru 1x2 a która 2x1, bardzo czytelne zadanie :)

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

2x2

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

2x1

$\mathbf{e}=(x^2,x^2-x)$

2x1

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

2x1

In [107]:
def B_ni_1(prop):
    random_value = np.random.uniform(0,1)

    if random_value < prop:
        return -1
    if random_value < 2*prop:
        return 1
    
    return 0

def print_matrix(matrix):
    if not matrix.all():
        print("The matrix is empty.")
        return
    
    column_widths = [max(len(str(item)) for item in col) for col in zip(*matrix)]
    
    total_width = sum(column_widths) + len(column_widths) * 3 + 1 
    horizontal_line = "+" + "-" * (total_width - 2) + "+"  

    lines = [horizontal_line]  
    for i, row in enumerate(matrix):
        formatted_row = " | ".join(str(item).ljust(column_widths[i]) for i, item in enumerate(row))
        lines.append(f"| {formatted_row} |")
        if i == len(matrix) -1:
            continue
        lines.append(horizontal_line)
    
    lines.append(horizontal_line)

    return "\n".join(lines)

def generate_s():
    s = np.array([ZnPoly([B_ni_1(prop) for _ in range(n)]) for _ in range(K)])
    s = s[np.newaxis]
    return s.T

def key_gen():
    def generate_A():
        return np.array([[ZnPoly(np.random.randint(0,q,size=n).tolist()) for _ in range(K)]
                                                         for _ in range(K)])
    def generate_e():
        return generate_s()
        

    def calculate_t(A,s,e):
        t = A @ s + e
        return t

    A = generate_A()
    s = generate_s()
    e = generate_e()
    t = calculate_t(A,s,e)

    return A,t,s

def test_calculate_t(A,s,e):
    t = A @ s + e
    return t


A = np.array([[ZnPoly([6,16,16,11]), ZnPoly([9,4,6,3])],
                [ZnPoly([5,3,10,1]),  ZnPoly([6,1,9,15])]])

s = np.array([ZnPoly([-1,-1,1,0]), ZnPoly([-1,0,-1,0])])
s = s[np.newaxis]
s = s.T

e = np.array([ZnPoly([0,1,0,0]), ZnPoly([0,1,-1,0])])
e = e[np.newaxis]
e = e.T

t = test_calculate_t(A,s,e)

print(f"s = {s} shape = {s.shape}")
print(f"t = {t} shape = {t.shape}")
print(f"A = \n{print_matrix(A)}\n shape = {A.shape}")


s = [[ 16x^3 + 16x^2 + x^1]
 [ 16x^3 + 16x^1]] shape = (2, 1)
t = [[ 16x^3 + 15x^2 + 7]
 [ 10x^3 + 12x^2 + 11x^1 + 6]] shape = (2, 1)
A = 
+------------------------------------------------------+
|  6x^3 + 16x^2 + 16x^1 + 11 |  9x^3 + 4x^2 + 6x^1 + 3 |
+------------------------------------------------------+
|  5x^3 + 3x^2 + 10x^1 + 1   |  6x^3 + x^2 + 9x^1 + 15 |
+------------------------------------------------------+
 shape = (2, 2)


### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

2x1

$\mathbf{e_1}=(x^2+x,x^2)$

2x1

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

2x1

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

1x1

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [108]:
A = np.array([  [ZnPoly([6,16,16,11]), ZnPoly([9,4,6,3])],
                [ZnPoly([5,3,10,1]),  ZnPoly([6,1,9,15])]])

t = test_calculate_t(A,s,e)

m = ZnPoly([1, 0, 1, 1])

r = np.array([ZnPoly([-1, 1, 0, 0]), ZnPoly([1, 1, 0, -1])])
r = r[np.newaxis]
r = r.T
    
e1 = np.array([ZnPoly([1, 1, 0]), ZnPoly([1, 0, 0])])
e1 = e1[np.newaxis]
e1 = e1.T

e2 = np.array([ZnPoly([-1, -1, 0, 0])])
e2 = e2[np.newaxis]
e2 = e2.T

#bob
def encrypt(A, t, m, e2, print_results = False, r=None, e1 = None):
    if r is None:
        r = generate_s()

    if e1 is None:
        e1 = generate_s()

    if print_results:
        print(f"A = \n{print_matrix(A)} \nshape = {A.shape}")
        print(f"t = {t} shape = {t.shape}")
        print(f"m = {m}")
        print(f"r = {r} shape = {r.shape}")

    u = A.T @ r + e1

    v = t.T @ r + e2 + [(ZnPoly([0,0,0,0,int(np.ceil(q/2))]) * m)][0]
    
    if print_results:  
        print(f"u = {u} shape = {u.shape}")
        print(f"v = {v} shape = {v.shape}")

    return [u,v]

c = encrypt(A,t,m, e2, True, r, e1)

test_c = [  np.array([ZnPoly([11,11,10,3]), ZnPoly([4,4,13,11])], dtype=object),
            np.array([ZnPoly([8,6,9,16])], dtype=object)]

assert c[0][0] == test_c[0][0]
assert c[0][1] == test_c[0][1]
assert c[1][0] == test_c[1][0]


A = 
+------------------------------------------------------+
|  6x^3 + 16x^2 + 16x^1 + 11 |  9x^3 + 4x^2 + 6x^1 + 3 |
+------------------------------------------------------+
|  5x^3 + 3x^2 + 10x^1 + 1   |  6x^3 + x^2 + 9x^1 + 15 |
+------------------------------------------------------+ 
shape = (2, 2)
t = [[ 16x^3 + 15x^2 + 7]
 [ 10x^3 + 12x^2 + 11x^1 + 6]] shape = (2, 1)
m =  x^3 + x^1 + 1
r = [[ 16x^3 + x^2]
 [ x^3 + x^2 + 16]] shape = (2, 1)
u = [[ 11x^3 + 11x^2 + 10x^1 + 3]
 [ 4x^3 + 4x^2 + 13x^1 + 11]] shape = (2, 1)
v = [[ 8x^3 + 6x^2 + 9x^1 + 16]] shape = (1, 1)


### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [109]:
def map_round(item, q = 17):
    middle = int(np.ceil(q/2))

    dist_to_middle = abs(item - middle)

    if abs(item - 0) > dist_to_middle and abs(item - q) > dist_to_middle:
        return 1
    else:
        return 0

#alice
def decrypt(c,s):
    def round_result(m_n):
        for i in range(len(m_n)):
            m_n.poly[i] = map_round(m_n.poly[i])
        return m_n
    
    u, v = c

    tmp = s.T @ u
    m_n = (v[0] - tmp[0])[0]

    return round_result(m_n)

m_n = decrypt(c, s)

print(f"m_n = {m_n}")

m_n =  x^3 + x^1 + 1


### Testy

In [116]:
import secrets as sc

success = 0
for i in range(1000):
    #e1 gen in key_gen()
    A,t,s = key_gen()

    #shouldn't the errors be also new?
    e2 = generate_s()
    
    m = [sc.choice((0,1)) for _ in range(4)]
    
    c = encrypt(A,t,m, e2)
    m_n = decrypt(c,s)

    if m_n.poly.get_coef(3) == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')


Success rate: 68.3 %
